# Tree Methods Consulting Project

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('puppies').getOrCreate()

In [5]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.sql.functions import isnan, when, count, col, isnull
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [6]:
#Read the data
data = spark.read.csv('/FileStore/tables/dog_food-1.csv', inferSchema=True, header=True)
data.printSchema()

root
-- A: integer (nullable = true)
-- B: integer (nullable = true)
-- C: double (nullable = true)
-- D: integer (nullable = true)
-- Spoiled: double (nullable = true)

In [7]:
#Explore the dataset
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
summary| A| B| C| D| Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
 count| 490| 490| 490| 490| 490|
 mean| 5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
 stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
 min| 1| 1| 5.0| 1| 0.0|
 max| 10| 10| 14.0| 10| 1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+

In [8]:
data.show()

+---+---+----+---+-------+
 A| B| C| D|Spoiled|
+---+---+----+---+-------+
 4| 2|12.0| 3| 1.0|
 5| 6|12.0| 7| 1.0|
 6| 2|13.0| 6| 1.0|
 4| 2|12.0| 1| 1.0|
 4| 2|12.0| 3| 1.0|
 10| 3|13.0| 9| 1.0|
 8| 5|14.0| 5| 1.0|
 5| 8|12.0| 8| 1.0|
 6| 5|12.0| 9| 1.0|
 3| 3|12.0| 1| 1.0|
 9| 8|11.0| 3| 1.0|
 1| 10|12.0| 3| 1.0|
 1| 5|13.0| 10| 1.0|
 2| 10|12.0| 6| 1.0|
 1| 10|11.0| 4| 1.0|
 5| 3|12.0| 2| 1.0|
 4| 9|11.0| 8| 1.0|
 5| 1|11.0| 1| 1.0|
 4| 9|12.0| 10| 1.0|
 5| 8|10.0| 9| 1.0|
+---+---+----+---+-------+
only showing top 20 rows

In [9]:
data.select([count(when(isnan(c), c)).alias(c) for c in data.columns]).show()

+---+---+---+---+-------+
 A| B| C| D|Spoiled|
+---+---+---+---+-------+
 0| 0| 0| 0| 0|
+---+---+---+---+-------+

In [10]:
data.select([count(when(isnull(c), c)).alias(c) for c in data.columns]).show()

+---+---+---+---+-------+
 A| B| C| D|Spoiled|
+---+---+---+---+-------+
 0| 0| 0| 0| 0|
+---+---+---+---+-------+

In [11]:
data.columns

Out[11]: ['A', 'B', 'C', 'D', 'Spoiled']

In [12]:
assembler = VectorAssembler(inputCols = ['A', 'B', 'C', 'D'], outputCol='features')

In [13]:
output = assembler.transform(data)
output.show()

+---+---+----+---+-------+-------------------+
 A| B| C| D|Spoiled| features|
+---+---+----+---+-------+-------------------+
 4| 2|12.0| 3| 1.0| [4.0,2.0,12.0,3.0]|
 5| 6|12.0| 7| 1.0| [5.0,6.0,12.0,7.0]|
 6| 2|13.0| 6| 1.0| [6.0,2.0,13.0,6.0]|
 4| 2|12.0| 1| 1.0| [4.0,2.0,12.0,1.0]|
 4| 2|12.0| 3| 1.0| [4.0,2.0,12.0,3.0]|
 10| 3|13.0| 9| 1.0|[10.0,3.0,13.0,9.0]|
 8| 5|14.0| 5| 1.0| [8.0,5.0,14.0,5.0]|
 5| 8|12.0| 8| 1.0| [5.0,8.0,12.0,8.0]|
 6| 5|12.0| 9| 1.0| [6.0,5.0,12.0,9.0]|
 3| 3|12.0| 1| 1.0| [3.0,3.0,12.0,1.0]|
 9| 8|11.0| 3| 1.0| [9.0,8.0,11.0,3.0]|
 1| 10|12.0| 3| 1.0|[1.0,10.0,12.0,3.0]|
 1| 5|13.0| 10| 1.0|[1.0,5.0,13.0,10.0]|
 2| 10|12.0| 6| 1.0|[2.0,10.0,12.0,6.0]|
 1| 10|11.0| 4| 1.0|[1.0,10.0,11.0,4.0]|
 5| 3|12.0| 2| 1.0| [5.0,3.0,12.0,2.0]|
 4| 9|11.0| 8| 1.0| [4.0,9.0,11.0,8.0]|
 5| 1|11.0| 1| 1.0| [5.0,1.0,11.0,1.0]|
 4| 9|12.0| 10| 1.0|[4.0,9.0,12.0,10.0]|
 5| 8|10.0| 9| 1.0| [5.0,8.0,10.0,9.0]|
+---+---+----+---+-------+-------------------+
only showing top 20 rows

In [14]:
final_data = output.select('features', 'Spoiled')

In [15]:
#Create 3 tree models

dtc = DecisionTreeClassifier(featuresCol='features', labelCol='Spoiled')

rfc = RandomForestClassifier(featuresCol='features', labelCol='Spoiled')

gbt = GBTClassifier(featuresCol='features', labelCol='Spoiled')

In [16]:
#Fit the 3 models on the full dataset

dtc_model = dtc.fit(final_data)
rfc_model = rfc.fit(final_data)
gbt_model = gbt.fit(final_data)

# Model Comparison

In [18]:
dtc_preds = dtc_model.transform(final_data)
rfc_preds = rfc_model.transform(final_data)
gbt_preds = gbt_model.transform(final_data)

In [19]:
#Evaluation metrics
acc_evaluator = MulticlassClassificationEvaluator(labelCol="Spoiled", predictionCol="prediction", metricName="accuracy")

In [20]:
dtc_acc = acc_evaluator.evaluate(dtc_preds)
rfc_acc = acc_evaluator.evaluate(rfc_preds)
gbt_acc = acc_evaluator.evaluate(gbt_preds)

In [21]:
print("Here are the results!")
print('-'*80)
print('A single decision tree had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*80)
print('A random forest ensemble had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*80)
print('An ensemble using GBT had an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!
--------------------------------------------------------------------------------
A single decision tree had an accuracy of: 98.57%
--------------------------------------------------------------------------------
A random forest ensemble had an accuracy of: 98.78%
--------------------------------------------------------------------------------
An ensemble using GBT had an accuracy of: 99.59%

# Feature importance

In [23]:
rfc_model.featureImportances

Out[40]: SparseVector(4, {0: 0.0189, 1: 0.0141, 2: 0.9384, 3: 0.0287})